# Brain Region Volume Computation

To run this, you need to download the Allen SDK https://allensdk.readthedocs.io/en/latest/

I stole the majority of this notebook from https://allensdk.readthedocs.io/en/latest/_static/examples/nb/reference_space.html. This shows how to grab a StructureTree (which is basically a tree of all the connected regions of the brain), select all the areas of the region your interested in (like the hypothalamus), use the "annotations" (whatever that means) to pull voxel data from each region, then compute the volume of each region based on the number of voxels and the resolution of the annotation.

### Constructing a StructureTree

Use this thing to get a list of all the nodes in the brain so we can grab their data.

In [174]:
from allensdk.api.queries.ontologies_api import OntologiesApi
from allensdk.core.structure_tree import StructureTree

oapi = OntologiesApi()
structure_graph = oapi.get_structures_with_sets([1])  # 1 is the id of the adult mouse structure graph

# This removes some unused fields returned by the query
structure_graph = StructureTree.clean_structures(structure_graph)  

tree = StructureTree(structure_graph)

In [175]:
# now let's take a look at a structure
tree.get_structures_by_name(['Hypothalamus'])  # ID = 1097

[{'acronym': 'HY',
  'rgb_triplet': [230, 68, 56],
  'graph_id': 1,
  'graph_order': 715,
  'id': 1097,
  'name': 'Hypothalamus',
  'structure_id_path': [997, 8, 343, 1129, 1097],
  'structure_set_ids': [2,
   112905828,
   691663206,
   12,
   184527634,
   112905813,
   687527670,
   114512891,
   114512892]}]

The fields are:
    * acronym: a shortened name for the structure
    * rgb_triplet: each structure is assigned a consistent color for visualizations
    * graph_id: the structure graph to which this structure belongs
    * graph_order: each structure is assigned a consistent position in the flattened graph
    * id: a unique integer identifier
    * name: the full name of the structure
    * structure_id_path: traces a path from the root node of the tree to this structure
    * structure_set_ids: the structure belongs to these predefined groups

We want to get all the descendants of the hypothalamus, which are all the nodes contained within it.

In [176]:
import pandas as pd
hy_descendants = tree.descendants([1097])  # Get all nodes in hypothalamus and all their nodes
hy_nodes_dict = {}  # Store all hypothalamus nodes with names, IDs, and volumes (which we'll grab later)
for d in hy_descendants[0]:
    hy_nodes_dict[d['name']] = d['id']
hy_descendants
hy_nodes = pd.DataFrame(list(hy_nodes_dict.items()), columns = ['Name', 'ID'])  # Convert dict to dataframe to make it nice
hy_nodes

,Name,ID
0,Hypothalamus,1097
1,Periventricular zone,157
2,Supraoptic nucleus,390
3,Accessory supraoptic group,332
4,Nucleus circularis,432
...,...,...
86,Tuberal nucleus,614
87,Zona incerta,797
88,Dopaminergic A13 group,796
89,Fields of Forel,804


### Downloading annotations data
I grabbed the highest resolution data, I think (25 microns)

In [177]:
import os
import nrrd
from allensdk.api.queries.mouse_connectivity_api import MouseConnectivityApi
from allensdk.config.manifest import Manifest

# the annotation download writes a file, so we will need somwhere to put it
annotation_dir = 'annotation'
Manifest.safe_mkdir(annotation_dir)

annotation_path = os.path.join(annotation_dir, 'annotation.nrrd')

# this is a string which contains the name of the latest ccf version
annotation_version = MouseConnectivityApi.CCF_VERSION_DEFAULT

resolution = 25  # I think 25 is the smallest resolution?
mcapi = MouseConnectivityApi()
mcapi.download_annotation_volume(annotation_version, resolution, annotation_path)

annotation, meta = nrrd.read(annotation_path)

2020-05-24 22:55:03,761 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://download.alleninstitute.org/informatics-archive/current-release/mouse_ccf/annotation/ccf_2017/annotation_25.nrrd


## Grabbing volume data
Here I assumed that the total number of voxels for a brain region represents the total volume of that region.

Total Volume
: The volume of that entire brain region, including all its subregions.

Direct volume
: The volume of the brain region, omitting subregions.

So if the hypothalamus is composed of only the dorsomedial nucleus (eg 10 mm<sup>3</sup>) and the preoptic nucleus (eg 35 mm<sup>3</sup>) and the hypothalmus no volume other than in those two regions, then the direct volume of the hypothalamus is 0 mm<sup>3</sup>, and the total volume is 45 mm<sup>3</sup>.

In [178]:
from allensdk.core.reference_space import ReferenceSpace

# build a reference space from a StructureTree and annotation volume, the third argument is 
# the resolution of the space in microns
rsp = ReferenceSpace(tree, annotation, [resolution, resolution, resolution])

In [179]:
rsp.direct_voxel_counts()
direct_voxel_map = rsp.direct_voxel_map
direct_voxel_map

{997: 227252,
 8: 0,
 567: 0,
 688: 0,
 695: 0,
 315: 0,
 184: 0,
 68: 16219,
 667: 14978,
 526157192: 23779,
 526157196: 7184,
 526322264: 67,
 500: 0,
 107: 0,
 219: 0,
 299: 0,
 644: 0,
 947: 0,
 985: 0,
 320: 85116,
 943: 236101,
 648: 209457,
 844: 184072,
 882: 11852,
 993: 0,
 656: 148747,
 962: 243488,
 767: 289459,
 1021: 152198,
 1085: 4786,
 453: 0,
 12993: 0,
 12994: 0,
 12995: 0,
 12996: 0,
 12997: 0,
 12998: 0,
 322: 0,
 793: 0,
 346: 0,
 865: 0,
 921: 0,
 686: 0,
 719: 0,
 353: 0,
 558: 25941,
 838: 40982,
 654: 35076,
 702: 36290,
 889: 51512,
 929: 3432,
 329: 0,
 981: 53686,
 201: 97044,
 1047: 82089,
 1070: 77323,
 1038: 83045,
 1062: 9389,
 480149202: 0,
 480149206: 0,
 480149210: 0,
 480149214: 0,
 480149218: 0,
 480149222: 0,
 480149226: 0,
 337: 0,
 1030: 19151,
 113: 41382,
 1094: 17474,
 1128: 35564,
 478: 33771,
 510: 3223,
 345: 0,
 878: 50090,
 657: 95921,
 950: 60615,
 974: 77950,
 1102: 105706,
 2: 7339,
 369: 0,
 450: 29562,
 854: 62626,
 577: 33490,
 625

In [180]:
tree.get_structures_by_id([997])  # 997 is the root a.k.a. the entire brain

[{'acronym': 'root',
  'rgb_triplet': [255, 255, 255],
  'graph_id': 1,
  'graph_order': 0,
  'id': 997,
  'name': 'root',
  'structure_id_path': [997],
  'structure_set_ids': [691663206]}]

In [181]:
rsp.total_voxel_counts()
total_voxel_map = rsp.total_voxel_map
total_voxel_map

{997: 32387385,
 8: 28774455,
 567: 17656111,
 688: 14179177,
 695: 13609203,
 315: 7891055,
 184: 62227,
 68: 16219,
 667: 14978,
 526157192: 23779,
 526157196: 7184,
 526322264: 67,
 500: 1565276,
 107: 0,
 219: 0,
 299: 0,
 644: 0,
 947: 0,
 985: 726598,
 320: 85116,
 943: 236101,
 648: 209457,
 844: 184072,
 882: 11852,
 993: 838678,
 656: 148747,
 962: 243488,
 767: 289459,
 1021: 152198,
 1085: 4786,
 453: 2133260,
 12993: 0,
 12994: 0,
 12995: 0,
 12996: 0,
 12997: 0,
 12998: 0,
 322: 1555772,
 793: 0,
 346: 0,
 865: 0,
 921: 0,
 686: 0,
 719: 0,
 353: 193233,
 558: 25941,
 838: 40982,
 654: 35076,
 702: 36290,
 889: 51512,
 929: 3432,
 329: 402576,
 981: 53686,
 201: 97044,
 1047: 82089,
 1070: 77323,
 1038: 83045,
 1062: 9389,
 480149202: 0,
 480149206: 0,
 480149210: 0,
 480149214: 0,
 480149218: 0,
 480149222: 0,
 480149226: 0,
 337: 150565,
 1030: 19151,
 113: 41382,
 1094: 17474,
 1128: 35564,
 478: 33771,
 510: 3223,
 345: 397621,
 878: 50090,
 657: 95921,
 950: 60615,
 9

Compute the total volume (in cubic microns) of the entire brain to validate:

In [182]:
total_voxel_map[997] * pow(resolution, 3)  # Num of voxels times resolution^3 to get total volume of brain

506052890625

We find the total brain volume is ~506 mm<sup>3</sup> which apparently is about the size of a mouse brain.

In [183]:
hy_nodes['Total Volume'] = hy_nodes.apply(lambda x: total_voxel_map[x['ID']] * pow(resolution, 3), axis=1)
hy_nodes['Direct Volume'] = hy_nodes.apply(lambda x: direct_voxel_map[x['ID']] * pow(resolution, 3), axis=1)
hy_nodes

,Name,ID,Total Volume,Direct Volume
0,Hypothalamus,1097,15196156250,2756718750
1,Periventricular zone,157,778187500,0
2,Supraoptic nucleus,390,46843750,46843750
3,Accessory supraoptic group,332,6062500,6062500
4,Nucleus circularis,432,0,0
...,...,...,...,...
86,Tuberal nucleus,614,523437500,523437500
87,Zona incerta,797,1800703125,1559468750
88,Dopaminergic A13 group,796,0,0
89,Fields of Forel,804,241234375,241234375


I'm assuming that the volumes for some of these are 0 because their size is < the resolution (25 microns).

All the subnodes direct volume + the hypothalamus direct volume should equal the total volume:

In [184]:
sum(hy_nodes.loc[0:]['Direct Volume']) == hy_nodes.loc[0]['Total Volume']

True

In [186]:
hy_nodes.to_csv('hypothalamus volumes.csv', index=False)